#### Steps
- import the CSV file into a pandas df
- clean the table name and remove all extra symbols, spaces, capital letters
- clean the column headers and remove all extra symbols, spaces, capital letters
- write the create table SQL statement
- import the data into the db

In [88]:
#import libraries
#conda install psycopg2
#pip install psycopg2

import pandas as pd
import numpy as np
import psycopg2 as pdb

In [89]:
!ls

Customer Contracts$.csv  Customer Engagements.csv py-automate.ipynb
Customer Demo.csv        py-automate              requirements.txt


In [90]:
df = pd.read_csv("Customer Contracts$.csv")
df.head()

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


In [91]:
#Clean table names
# lower case letters
# remove all white spaces and $
# replace -, /, // with _
file = "Customer Contracts$"

clean_tbl_name = file.lower().replace(" ", "_").replace("?", "") \
    .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "") \
    .replace(")", "").replace(r"(", "").replace("$", "")
clean_tbl_name

'customer_contracts'

In [92]:
#Clean header names
# lower case letters
# remove all white spaces and $
# replace -, /, // with _
df.columns = [x.lower().replace(" ", "_").replace("?", "") \
    .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "") \
    .replace(")", "").replace(r"(", "").replace("$", "") for x in df.columns]

df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [93]:
"""
create table customer_contracts
(
    customer_name       varchar,
    start_date          varchar,
    end_date            varchar,
    contract_amount_m   float,
    invoice_sent        varchar,
    paid                varchar,
);
"""

'\ncreate table customer_contracts\n(\n    customer_name       varchar,\n    start_date          varchar,\n    end_date            varchar,\n    contract_amount_m   float,\n    invoice_sent        varchar,\n    paid                varchar,\n);\n'

In [94]:
df.dtypes

customer_name         object
start_date            object
end_date              object
contract_amount_m    float64
invoice_sent          object
paid                  object
dtype: object

In [95]:
replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [96]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar'

In [97]:
# #Enter the values for your database connection
# dsn_database = "pyautomation"
# dsn_hostname = "192.168.1.10"
# dsn_port = "5432"
# dsn_uid = "root"
# dsn_pwd = "secret"

In [98]:
# #open a database connection
# conn_string = "host="+dsn_hostname+" port="+dsn_port+" dbname="+dsn_database+" user="+dsn_uid+"password="+dsn_pwd
#
# conn = psycopg2.connect(conn_string)
# cursor = conn.cursor()
# print('opened database successfully')

In [99]:
conn_string = "host='localhost' dbname='pyautomation' user='root' password='secret'"
print ("Connecting to database\n	->%s" % (conn_string))

conn = pdb.connect(conn_string)
cursor = conn.cursor()
print('opened database successfully')

Connecting to database
	->host='localhost' dbname='pyautomation' user='root' password='secret'
opened database successfully


In [100]:
clean_tbl_name

'customer_contracts'

In [101]:
#drop tables with same name
cursor.execute("drop table if exists customer_contracts;")


In [102]:
col_str

'customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar'

In [103]:
# creat table
cursor.execute("create table customer_contracts (customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar)")


In [104]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' ")

for table in cursor.fetchall():
    print(table)

('customer_contracts',)


In [106]:
#insert values to table

#save df to csv
df.to_csv('customer_contracts.csv', header=df.columns, index=False, encoding='utf-8')

#open the csv file, save it as object
my_file = open('customer_contracts.csv')
print('file opened in memory')

file opened in memory


In [107]:
#upload to db
SQL_STATEMENT = """
COPY customer_contracts FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [108]:
cursor.execute("grant select on table customer_contracts to public")
conn.commit()

cursor.close()
print('table customer_contracts imported to db completed')

table customer_contracts imported to db completed
